Preprocessing
===



In [ ]:
from constants import *  # bad practice but YOLO
import preproc

In [ ]:
import constants
reload(constants);
from constants import *

In [ ]:
reload(preproc);
y_tr, x_tr = preproc.read_data(TRAIN_FILE, #filename
                              'subreddit', #label field
                              preprocessor=preproc.tokenize_and_downcase) #your preprocessor

y_te, x_te = preproc.read_data(TEST_FILE, #filename
                              'subreddit', #label field
                              preprocessor=preproc.tokenize_and_downcase) #your preprocessor

In [ ]:
reload(preproc);
# vocab = preproc.create_vocab(x_tr, x_te)
counts = preproc.get_vocab_counts(x_tr, x_te)
vocab = [w for w, _ in counts.most_common(6000)]
vocab.append(OFFSET)
K = len(vocab)

In [ ]:
reload(preproc);
mapping = preproc.create_mapping(vocab, K)

In [ ]:
# Now, we'll turn all of our input features into feature vectors
import numpy as np

Y_tr = np.array(y_tr)
X_tr = [preproc.feature_vector(bow, vocab, K, mapping) for bow in x_tr]

Y_te = np.array(y_te)
X_te = [preproc.feature_vector(bow, vocab, K, mapping) for bow in x_te]

Learning
=========

In [141]:
import learner

First, let's try logistic regression

In [161]:
%%time
reload(learner);
lr = learner.trainLR(X_tr, Y_tr)

CPU times: user 19.2 s, sys: 27.2 ms, total: 19.2 s
Wall time: 19.3 s


In [163]:
print learner.accuracy(lr, X_tr, Y_tr)
print learner.accuracy(lr, X_te, Y_te)

0.99975
0.722


Now, let's try a neural network

In [162]:
%%time
reload(learner);
mlp = learner.trainMLP(X_tr, Y_tr)

CPU times: user 1min 19s, sys: 157 ms, total: 1min 19s
Wall time: 1min 19s


In [164]:
print learner.accuracy(mlp, X_tr, Y_tr)
print learner.accuracy(mlp, X_te, Y_te)

1.0
0.696


Now, let's try Naive Bayes

In [169]:
%%time
reload(learner);
nb = learner.trainNB(X_tr, Y_tr)

CPU times: user 319 ms, sys: 73.8 ms, total: 393 ms
Wall time: 394 ms


In [170]:
print learner.accuracy(nb, X_tr, Y_tr)
print learner.accuracy(nb, X_te, Y_te)

0.9365
0.64


# Dimensionality Reduction

We've had some pretty slow training times for LR and NN. Can we do better?


In [174]:
%%time
reload(learner);
# Random Projections
rp_tr, rp_te = learner.get_RP_features(X_tr, X_te, n_components=4000)

CPU times: user 10.2 s, sys: 101 ms, total: 10.3 s
Wall time: 10.3 s


Logistic Regression with RP

In [160]:
%%time
lr_RP = learner.trainLR(rp_tr, Y_tr)


CPU times: user 12.8 s, sys: 15.3 ms, total: 12.8 s
Wall time: 12.9 s


In [167]:
print learner.accuracy(lr_RP, rp_tr, Y_tr)
print learner.accuracy(lr_RP, rp_te, Y_te)

0.99975
0.684


MLP with RP

In [165]:
%%time
mlp_rp = learner.trainMLP(rp_tr, Y_tr)

CPU times: user 54.9 s, sys: 48.8 ms, total: 54.9 s
Wall time: 55 s


In [166]:
print learner.accuracy(mlp_rp, rp_tr, Y_tr)
print learner.accuracy(mlp_rp, rp_te, Y_te)

1.0
0.632


NB with RP

In [171]:
%%time
nb_rp = learner.trainNB(rp_tr, Y_tr)

CPU times: user 220 ms, sys: 33 ms, total: 253 ms
Wall time: 256 ms


In [172]:
print learner.accuracy(nb_rp, rp_tr, Y_tr)
print learner.accuracy(nb_rp, rp_te, Y_te)

0.44025
0.398
